In [1]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
!pip install sentence-transformers
!pip install -U sentence-transformers
!pip install numpy pandas scikit-learn matplotlib seaborn umap-learn
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.7 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [3]:
!unzip /content/AI_Organized_Images.zip -d ./images/

Archive:  /content/AI_Organized_Images.zip
   creating: ./images/AI_Organized_Images/
   creating: ./images/AI_Organized_Images/Group_0_vehicle/
  inflating: ./images/AI_Organized_Images/Group_0_vehicle/car_0000.jpg  
  inflating: ./images/AI_Organized_Images/Group_0_vehicle/car_0001.jpg  
  inflating: ./images/AI_Organized_Images/Group_0_vehicle/car_0002.jpg  
  inflating: ./images/AI_Organized_Images/Group_0_vehicle/car_0003.jpg  
  inflating: ./images/AI_Organized_Images/Group_0_vehicle/car_0004.jpg  
  inflating: ./images/AI_Organized_Images/Group_0_vehicle/car_0005.jpg  
  inflating: ./images/AI_Organized_Images/Group_0_vehicle/car_0006.jpg  
  inflating: ./images/AI_Organized_Images/Group_0_vehicle/car_0007.jpg  
  inflating: ./images/AI_Organized_Images/Group_0_vehicle/car_0008.jpg  
  inflating: ./images/AI_Organized_Images/Group_0_vehicle/car_0009.jpg  
  inflating: ./images/AI_Organized_Images/Group_0_vehicle/car_0010.jpg  
  inflating: ./images/AI_Organized_Images/Group_0_ve

In [4]:
import nltk


# Download 'averaged_perceptron_tagger' for POS tagging
nltk.download('averaged_perceptron_tagger')

# Download 'words' for word checking
nltk.download('words')
!pip install spacy
!python -m spacy download en_core_web_sm

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 79.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
import glob
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
import spacy
from collections import Counter
import os
# Load the SpaCy model for English
nlp = spacy.load("en_core_web_sm")

def is_valid_sentence(caption):
    """Check if the caption has at least one noun and one verb and is long enough."""
    doc = nlp(caption)
    has_noun = any(token.pos_ == 'NOUN' for token in doc)
    has_verb = any(token.pos_ == 'VERB' for token in doc)
    return len(doc) > 3 and has_noun and has_verb

# Load the BLIP model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
# for 10 images using 3 min ,for all images using 1.5 hrs dont modify this 10-20 is okay
# according to statistics the sample size of 15 is enought to represent the 80% of the real sample
def generate_captions(image_paths, max_images=10):
    """Generate captions for the first few images."""
    captions = []
    # Limit to the first max_images paths
    limited_image_paths = image_paths[:max_images]

    for image_path in limited_image_paths:
        # Load the image using PIL
        image = Image.open(image_path)
        # Process the image
        inputs = processor(images=image, return_tensors="pt")
        # Generate the caption
        output = model.generate(**inputs)
        caption = processor.decode(output[0], skip_special_tokens=True)
        captions.append(caption)

    return captions

def generate_group_name(captions):
    """Generate a suitable new group name based on captions."""
    all_nouns = []
    all_verbs = []

    for caption in captions:
        doc = nlp(caption)
        nouns = [token.lemma_ for token in doc if token.pos_ == 'NOUN']
        verbs = [token.lemma_ for token in doc if token.pos_ == 'VERB']
        all_nouns.extend(nouns)
        all_verbs.extend(verbs)

    common_noun = Counter(all_nouns).most_common(1)
    common_verb = Counter(all_verbs).most_common(1)
    new_name = f"{common_noun[0][0] if common_noun else 'Group'}_{common_verb[0][0] if common_verb else 'Activity'}"

    return new_name

def ensure_unique_name(new_name, existing_names):
    """Ensure the new name for the organized folder is unique."""
    count = 1
    base_name = new_name
    while new_name in existing_names:
        new_name = f"{base_name}_{count}"
        count += 1
    return new_name


def process_image_groups(base_dir):
    """Process image groups by generating captions and renaming directories."""
    new_group_names = {}
    existing_group_names = set()

    for group_folder in os.listdir(base_dir):
        group_path = os.path.join(base_dir, group_folder)
        if os.path.isdir(group_path):
            image_paths = glob.glob(os.path.join(group_path, "*.jpg")) + glob.glob(os.path.join(group_path, "*.png"))
            if not image_paths:
                print(f"No images found in {group_path}. Skipping.")
                continue

            # Generate captions, limiting to the top 10 images (or fewer if not available)
            captions = generate_captions(image_paths)

            # Filter valid captions
            valid_captions = [caption for caption in captions if is_valid_sentence(caption)]
            if valid_captions:
                new_name = generate_group_name(valid_captions)
                new_name = ensure_unique_name(new_name, existing_group_names)

                # Rename the group folder
                new_folder_path = os.path.join(base_dir, new_name)
                os.rename(group_path, new_folder_path)

                new_group_names[group_folder] = new_name
                existing_group_names.add(new_name)
            else:
                print(f"No valid captions generated for {group_folder}.")

    print(new_group_names)

# Set your base directory of groups
base_directory = "./images/AI_Organized_Images"
process_image_groups(base_directory)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'Group_0_vehicle': 'car_drive', 'Group_2_object': 'jet_fly', 'Group_1_portrait': 'cat_sit'}
